In [ ]:
!pip install --upgrade openai
!pip install xlrd
!pip install -U sentence-transformers

In [2]:
import numpy as np
dataset = np.load('/content/WP-train.npy', allow_pickle=True)
answers = []
questions = []
choices = []
for i in range(len(dataset)):
  answers.append(dataset[i]["answer"])
  questions.append(dataset[i]["question"])
  choices.append(dataset[i]["choice_list"])

In [ ]:
for i in range(0,10):
  print(questions[i], answers[i])

In [ ]:
from openai import OpenAI
import math
from numpy.linalg import norm
import time
client = OpenAI(api_key='API_KEY')
import openpyxl
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

wb = openpyxl.load_workbook('/content/EncodedQuestions.xlsx')
ws = wb.active

def findRiddle(Riddle):
  question_emb = model.encode(Riddle)
  A= question_emb
  norm_A = norm(A)
  highest = 0
  for i in range(1,ws.max_row):
    B = [float(x) for x in ws["C"+str(i)].value.split(",")]
    sim = np.dot(A,B)/(norm_A*norm(B))
    if sim > highest:
        highest = sim
        nearest = i
  return [ws["A"+str(nearest)].value, ws["B"+str(nearest)].value]
def findRiddle2(Riddle, index):
  question_emb = model.encode(Riddle)
  A= question_emb
  norm_A = norm(A)
  highest = 0
  for i in range(1,ws.max_row):
    if i == index:
      time.sleep(10)
    else:
      B = [float(x) for x in ws["C"+str(i)].value.split(",")]
      sim = np.dot(A,B)/(norm_A*norm(B))
      if sim > highest:
          highest = sim
          nearest = i
  return [ws["A"+str(nearest)].value, ws["B"+str(nearest)].value]
def farRiddle(Riddle):
  question_emb = model.encode(Riddle)
  A= question_emb
  norm_A = norm(A)
  lowest = 1
  for i in range(1,ws.max_row):
    B = [float(x) for x in ws["C"+str(i)].value.split(",")]
    sim = np.dot(A,B)/(norm_A*norm(B))
    if sim < lowest:
        lowest = sim
        nearest = i
  return [ws["A"+str(nearest)].value, ws["B"+str(nearest)].value]
def topTwoRiddles(Riddle):
  question_emb = model.encode(Riddle)
  A= question_emb
  norm_A = norm(A)
  highest = 0
  secondhighest = 0
  secondnearest = 0
  for i in range(1,ws.max_row+1):
    B = [float(x) for x in ws["C"+str(i)].value.split(",")]
    sim = np.dot(A,B)/(norm_A*norm(B))
    if sim > highest:
        highest = sim
        nearest = i
    elif sim>secondhighest:
        secondhighest = sim
        secondnearest = i
  print(highest, nearest)
  return [ws["A"+str(nearest)].value, ws["B"+str(nearest)].value, ws["A"+str(secondnearest)].value, ws["B"+str(secondnearest)].value]


In [ ]:
print(questions[0])
print(choices[0])
print(findRiddle(questions[0]))
print(farRiddle(questions[0]))

In [ ]:
choices[10]

In [ ]:
#Check Baseline
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

correct = 0
count = 0
for x in range(0,len(questions)):
    #closeRiddle = findRiddle(questions[x])
    demonstration = "Please pick the best choice for the brain teaser. Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:"
    #prompt = str("Please pick the best choice for the brain teaser. Each brain teaser has only one possible solution, including the choice none of the above, answer should only provide the choice:" +str(choices[x][0]) +','+ str(choices[x][1])+','+ str(choices[x][2]))
    prompt = demonstration + "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3]
    completion = client.chat.completions.create(
      model="gpt-4-1106-preview",
      messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": prompt}
      ]
    )
    pred = completion.choices[0].message.content
    answer = ""
    val = similar(choices[x][0], pred)
    if val < similar(choices[x][1], pred):
      answer = choices[x][1]
      val = similar(choices[x][1],pred)
    if val < similar(choices[x][2], pred):
      answer = choices[x][2]
      val = similar(choices[x][2],pred)
    if val < similar(choices[x][3], pred):
      answer = choices[x][3]
      val = similar(choices[x][3],pred)
    #Calculate Accuracy
    if answer  == answers[x]:
      correct += 1
      print("Correct" + pred)
    else:
      print("Incorrect:" + pred)
    count += 1

print(correct, count, correct/count)

In [ ]:
#Check Baseline + Task Description
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

correct = 0
count = 0
for x in range(0,len(questions)):
    demonstration = "Please pick the best choice for the word puzzle brain teaser (Word-type brain teaser where the answer violates the default meaning of the word and focuses on the letter composition of the target question). Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:"
    prompt = demonstration + "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3]
    completion = client.chat.completions.create(
      model="gpt-4-1106-preview",
      messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": prompt}
      ]
    )
    pred = completion.choices[0].message.content
    answer = ""
    val = similar(choices[x][0], pred)
    if val < similar(choices[x][1], pred):
      answer = choices[x][1]
      val = similar(choices[x][1],pred)
    if val < similar(choices[x][2], pred):
      answer = choices[x][2]
      val = similar(choices[x][2],pred)
    if val < similar(choices[x][3], pred):
      answer = choices[x][3]
      val = similar(choices[x][3],pred)
    #Calculate Accuracy
    if answer  == answers[x]:
      correct += 1
    count += 1

print(correct, count, correct/count)

In [ ]:
#Check Baseline + Explain
from difflib import SequenceMatcher
import json
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

correct = 0
count = 0
for x in range(0,len(questions)):
    demonstration = "Please pick the best choice for the brain teaser. Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:"
    prompt = demonstration + "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3]
    completion = client.chat.completions.create(
      model="gpt-4-1106-preview",
      messages=[
        {"role": "system", "content": 'You are a riddle solving expert that explains solutions step by step and responds in raw JSON format following the format { "response": <answer>, "explanation": <explaination> }.'},
        {"role": "user", "content": prompt}
      ]
    )
    #print(completion['choices'][0]['message']['content'])
    pred = completion.choices[0].message.content
    try:
      pred = json.loads(pred)['response']
    except:
      pred = pred
    answer = choices[x][0]
    val = similar(choices[x][0], pred)
    if val < similar(choices[x][1], pred):
      answer = choices[x][1]
      val = similar(choices[x][1],pred)
    if val < similar(choices[x][2], pred):
      answer = choices[x][2]
      val = similar(choices[x][2],pred)
    if val < similar(choices[x][3], pred):
      answer = choices[x][3]
      val = similar(choices[x][3],pred)
    #Calculate Accuracy

    if answer  == answers[x]:
      correct += 1
      print("Correct")
    else:
      print("Wrong")

    count += 1

print(correct, count, correct/count)

In [ ]:
questions

In [ ]:
#Oneshot

from difflib import SequenceMatcher
import json
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

correct = 0
count = 0
x = 0
while x < 51:
    closeRiddle = findRiddle2(questions[x], x+1)
    #closeRiddle = farRiddle(questions[x])
    #demonstration = "Please pick the best choice for the following three brain teasers. Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice. consider the individual letters of the words as well as the sounds they make. For example, the letter C sounds like 'see', U sounds like 'you', and so on."
    demonstration = "Answer the following riddles by analyzing the language play, symbolic representation, or unconventional interpretation involved. Look for hidden meanings or alternate representations in the words or phrases used."
    prompt = demonstration +  "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3]
    prompt = prompt + "Question:" + questions[x+1]+ " \n Choice:" + "\n (A) " + choices[x+1][0] + "\n (B) " + choices[x+1][1] + "\n (C) " + choices[x+1][2] + "\n (D) " +choices[x+1][3]
    prompt = prompt +"Question:" + questions[x+2]+ " \n Choice:" + "\n (A) " + choices[x+2][0] + "\n (B) " + choices[x+2][1] + "\n (C) " + choices[x+2][2] + "\n (D) " +choices[x+2][3]
    prompt = prompt + "\n Example Questions with corresponding example answer:" +"\n" +closeRiddle[0] + "\n The answer would be" + closeRiddle[1]
    completion = client.chat.completions.create(
      model="gpt-4-1106-preview",
      messages=[
        {"role": "system", "content": 'You are a helpful assistant that just gives the answers to a set of three questions in raw JSON format following the format and not markdown so I can easily exctract the output { "question1": <answer>, "explanation1": <explanation>, "question2": <answer>, "explanation2": <explanation>, "question3": <answer>, "explanation3": <explanation>}.'},
        {"role": "user", "content": prompt}
      ]
    )
    pred = completion.choices[0].message.content
    try:
      pred = json.loads(pred)
      pred = [pred["question1"], pred["question2"], pred["question3"]]
    except:
      pred = pred
    for i in range(0,3):
      if "(A)" in pred[i]:
        answer = choices[x][0]
      elif "(B)" in pred[i]:
        answer = choices[x][1]
      elif "(C)" in pred[i]:
        answer = choices[x][2]
      elif "(D)" in pred[i]:
        answer = choices[x][3]
      else:
        answer = choices[x][0]
        val = similar(choices[x][0], pred[i])
        if val < similar(choices[x][1], pred[i]):
          answer = choices[x][1]
          val = similar(choices[x][1],pred[i])
        if val < similar(choices[x][2], pred[i]):
          answer = choices[x][2]
          val = similar(choices[x][2],pred[i])
        if val < similar(choices[x][3], pred[i]):
          answer = choices[x][3]
          val = similar("D",pred[i])
      #Calculate Accuracy
      if answer  == answers[x]:
        correct += 1
        print("Correct" + answer)
      else:
        print("Wrong" + "Prediction: " +  pred[i] + "Selected Answer" + answer + "Correct Answer" + answers[x])
      x = x + 1
    print("------------------")

print(correct, x, correct/x)

In [ ]:
pred

In [ ]:
#Oneshot Explain
from difflib import SequenceMatcher
import json
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

correct = 0
count = 0
for x in range(0,len(questions)):
    #closeRiddle = findRiddle(questions[x])
    closeRiddle = farRiddle(questions[x])
    demonstration = "Please pick the best choice for the brain teaser. Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:"
    prompt = demonstration + "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3] + "\n Example Questions with corresponding example answer:" +"\n" +closeRiddle[0] + "\n The answer would be" + closeRiddle[1]
    completion = client.chat.completions.create(
      model="gpt-4-1106-preview",
      messages=[
        {"role": "system", "content": 'You are a riddle solving expert that explains solutions step by step and responds in raw JSON format following the format { "response": <answer>, "explanation": <explaination> }.'},
        {"role": "user", "content": prompt}
      ]
    )
    #print(completion['choices'][0]['message']['content'])
    pred = completion.choices[0].message.content
    try:
      pred = json.loads(pred)['response']
    except:
      pred = pred
    answer = choices[x][0]
    val = similar("A", pred)
    if val < similar("B", pred):
      answer = choices[x][1]
      val = similar("B",pred)
    if val < similar("C", pred):
      answer = choices[x][2]
      val = similar("C",pred)
    if val < similar("D", pred):
      answer = choices[x][3]
      val = similar("D",pred)
    #Calculate Accuracy
    print(answer)
    if answer  == answers[x]:
      correct += 1


    count += 1

print(correct, count, correct/count)